sync linkedin contacts with google contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');
var glob = require('glob');
var path = require('path');
var {
    getContacts,
    runSeleniumCell
} = importer.import([
    'list google contacts',
    'selenium cell'
]);
var listAllConnections, scrapeEntireLinkedInProfile;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';
function escapeFilename(f) {
    return f.replace(/[^a-z0-9-\s]/ig, '_')
}

function syncLinkedInContacts() {
    var loaded = glob.sync('**/linkedin-*.json', {cwd: project})
        .reduce((arr, l) => {
            try {
                arr.push(escapeFilename(path.basename(l).replace('.json', '').replace('linkedin-', '')))
                arr.push(escapeFilename(JSON.parse(fs.readFileSync(path.join(project, l)))[0]
                    .url.replace(/^.*?\/*in\/|\/$/ig, '')));
            } catch (e) {
                console.log(l);
            }
            return arr;
        }, []);
    var fresh;
    return runSeleniumCell([
        'log in linkedin',
        'scrape linkedin profile',
        'scrape linkedin contacts',
        'scrape entire linkedin profile'
    ])
        .then(r => ({
            listAllConnections, scrapeEntireLinkedInProfile
        } = r).loginLinkedIn())
        .then(() => listAllConnections())
        .then((connections) => {
            fresh = connections.filter(c => {
                const fname = escapeFilename(c.replace(/^.*?\/*in\/|\/$/ig, ''));
                return loaded.indexOf(fname) === -1
                    && loaded.indexOf(c) === -1
            });
            const percent = Math
                .round((connections.length - fresh.length) / connections.length * 100);
            console.log((connections.length - fresh.length)
                + ' / ' + connections.length + ' : '
                + percent
                + '%');
        })
        .then(() => {
            const rand = Math.floor(fresh.length / 30 * Math.random()) * 30;
            console.log(fresh.slice(rand, rand + 30));
            return importer.runAllPromises(fresh.slice(rand, rand + 30).map(c => resolve => {
                var linkedIn;
                const fname = c.replace(/^\/*in\/|\/$/ig, '');
                return scrapeEntireLinkedInProfile('https://linkedin.com' + c)
                    .then(r => {
                        linkedIn = r;
                        return getContacts({
                            displayName: r.name.split(/\s/).join('.*')
                        });
                    })
                    .then(r => {
                        console.log(linkedIn.name);
                        if (r.length === 0) {
                            r.push({});
                        }
                        Object.assign(r[0], linkedIn);
                        fs.writeFileSync(
                            project + '/linkedin-' + escapeFilename(fname) + '.json',
                            JSON.stringify(r, null, 4));
                        return resolve(r);
                    })
                    .catch(e => {
                        console.log(e);
                        resolve()
                    })
            }))
        })
};

module.exports = syncLinkedInContacts;



scrape entire linkedin profile?



In [ ]:
var importer = require('../Core');

function loadEntirePage() {
    return client
    // load all defered elements by scrolling
        .then(() => getAllUntil(
            false,
            '//*[contains(@class, "pv-deferred-area")]/@id',
            [],
            (a, b) => a === b,
            (i) => i < 30
        ))
        .pause(2000)
        // show more all sections
        .scroll(0, 0)
        .then(() => importer.runAllPromises([
            '//button[contains(., "See more")]',
            '//button[contains(., "more skills")]',
            '//button[contains(., "more education")]',
            '//button[contains(., "more recommendations")]'
        ].map(selector => resolve => {
            return client
                .isVisible(selector)
                .then(is => is
                    ? client.click(selector).pause(1000)
                    : [])
                .then(e => resolve())
                .catch(e => {console.log; return resolve()})
        })))
        .catch(e => console.log(e))
};

function scrapeEntireLinkedInProfile(profile) {
    var contact;
    return client
        .then(() => clickSpa(profile))
        .then(() => readLinkedInProfileInfo())
        .then(r => {
            contact = r;
            return loadEntirePage();
        })
        // get extra profile info
        .then(() => {
            return getAllXPath({
                summary: '//p[contains(@class, "section__summary")]//text()',
                experience: [
                    '//*[contains(@class, "profile-section__card")]|//*[contains(@class, "profile-section__section-info-item")]',
                    ['.//text()']
                ],
                skills: [
                    '//*[contains(@class, "skill-entity--featured")]|//*[contains(@class, "skill-entity__pill")]',
                    ['.//text()']
                ],
                recommendations: [
                    '//*[contains(@class, "recommendation-entity")]',
                    ['.//text()']
                ],
                interests: [
                    '//*[contains(@class, "interest-entity")]',
                    ['.//text()']
                ]
            });
        })
        .then(r => {
            return Object.assign(r, {
                summary: r.summary.join(''),
                experience: r.experience.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                skills: r.skills.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                recommendations: r.recommendations.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/)),
                interests: r.interests.map(e => typeof e[0] === 'string'
                    ? e[0]
                    : e[0].join('')
                        .trim().split(/\s*\n+\s*/))
            }, contact)
        })
        .catch(e => console.log(e))
};
module.exports = scrapeEntireLinkedInProfile;



scrape linkedin contacts?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function listAllConnections(force = false) {
    var friends = [];
    if (fs.existsSync(project + '/connections.json') && !force
        && (new Date()).getTime() - fs.statSync(project + '/connections.json').mtime.getTime() < 1000 * 60 * 60 * 24) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/connections.json')));
    }
    console.log('fetching new linkedin contacts');
    return client
        .getUrl()
        .url('https://www.linkedin.com/mynetwork/invite-connect/connections/')
        .pause(3000)
        .then(() => getAllUntil(
            false,
            '//a[contains(@href, "/in/")]/@href',
            friends,
            (a, b) => a === b,
            (i) => i < 10
        ))
        .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
        .then(r => {
            fs.writeFileSync(
                project + '/connections.json',
                JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
};
module.exports = listAllConnections;


test linkedin scraper


In [ ]:
var importer = require('../Core');
var syncLinkedInContacts = importer.import('sync linkedin contacts');
$$.async();
syncLinkedInContacts()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



connect on linkedin?


In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Collections/conversations';

function listAllConnections(list) {
    var friends = [];
    // TODO: create utility function for managing time critical files like this.
    if (fs.existsSync(project + '/new-connections.json')) {
        return Promise.resolve(JSON.parse(
            fs.readFileSync(project + '/new-connections.json')));
    }
    return client
        .getUrl()
        .then(url => {
            if(list && list.includes('linkedin.com')) {
                return client
                    .url(list)
                    .pause(3000);
            }
            return client
                .url('https://www.linkedin.com/mynetwork/')
                .pause(3000);
        })
        .then(() => getAllUntil(
            false,
            '//a[contains(@href, "/in/")]/@href',
            friends,
            (a, b) => a === b,
            (i) => i < 10
        ))
        .then(r => r.filter((l, i, arr) => arr.indexOf(l) === i))
        .then(r => {
            fs.writeFileSync(
                project + '/new-connections.json',
                JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
};

function connectLinkedin(profile) {
    return client
        .then(() => clickSpa('https://linkedin.com' + profile))
        .pause(3000)
        .isExisting('//button[contains(., "Connect")]')
        .then(is => {
            console.log(is + ' - ' + profile);
            return is
              ? client
                  .isVisible('//button[contains(., "Connect")]')
                  .then(is => is
                        ? client
                            .click('//button[contains(., "Connect")]')
                            .pause(1000)
                        : [])
              : client
                  .isExisting('//button[contains(., "More")]')
                  .then(is => is ? client.click('//button[contains(., "More")]').pause(500) : [])
                  .isVisible('//button[contains(., "Connect")][not(contains(., "Remove"))]')
                  .then(is => is
                        ? client
                            .click('//button[contains(., "Connect")][not(contains(., "Remove"))]')
                            .pause(1000)
                        : [])
        })
        .isExisting('//button[contains(., "Send now")]')
        .then(is => is
              ? client
              .isVisible('//button[contains(., "Send now")]')
              .then(is => is
                    ? client
                        .click('//button[contains(., "Send now")]')
                        .pause(2000)
                    : [])
              : [])
        .pause(1000)
        .catch(e => console.log(e))
}

module.exports = {
    listAllConnections,
    connectLinkedin
};


connect add friends linkedin?


In [ ]:
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var runSeleniumCell = importer.import('run selenium cell');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';


var listAllConnections, connectLinkedin, loginLinkedIn;
function addLinkedinConnections(conn, list) {
    var pages;
    return runSeleniumCell(['log in linkedin', 'connect on linkedin'])
        .then(r => {
            listAllConnections = r[1].listAllConnections;
            connectLinkedin = r[1].connectLinkedin;
            loginLinkedIn = r.loginLinkedIn;
            return loginLinkedIn();
        })
        .then(() => typeof conn === 'string' ? [conn] : (typeof conn !== 'undefined' ? conn : listAllConnections(list)))
        .then(pages => {
            const rand = Math.round(pages.length / 30 * Math.random()) * 30;
            console.log(pages.slice(rand, rand + 30));
            return importer.runAllPromises(pages.slice(rand, rand + 30).map(page => resolve => {
                return connectLinkedin(page, true)
                    .then(() => resolve(page))
                    .catch(e => resolve(e))
            }));
        })
        .then(r => {
            console.log(r);
            return r;
        })
}
module.exports = addLinkedinConnections;



In [2]:
var importer = require('../Core');
var addLinkedinConnections = importer.import('connect add friends linkedin');

$$.async();
addLinkedinConnections(null, '')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


reading notebook /Users/briancullinan/jupytangular2/Selenium/linkedin connections.ipynb
compiling /Users/briancullinan/jupytangular2/Selenium/linkedin connections.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[0]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Selenium/webdriver.ipynb[3]
init - locking
init - lock
using existing session 0 - 9ec343774a8c83a31c38c779bd57c7a1
update - locking
update 9ec343774a8c83a31c38c779bd57c7a1
init - unlocking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
init - unlock
scanning on
update - unlock
update - locking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
[ '9ec34

update - unlocking
update - unlock
update - locking
false - /in/randycourtney/
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
false - /in/troygrissom/
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
true - /in/alexander-devine-61903540/
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update - unlock
update - locking
false - /in/tobie-madsen-96887454/
update - lock
update 9ec343774a8c83a31c38c779bd57c7a1
update - unlocking
update -

[ '/in/gpike/',
  '/in/belminamehmedagic/',
  '/in/dan-wilson-959b149/',
  '/in/rob-fay-23876422/',
  '/in/amanda-hathaway-87882225/',
  '/in/nikkipeckham/',
  '/in/justin-beaudry-05700164/',
  '/in/amberpechin/',
  '/in/steveshaw5/',
  '/in/grkosirog/',
  '/in/zachtullis/',
  '/in/sarahccolombo/',
  '/in/vanessa-greffe-59246025/',
  '/in/boboratory/',
  '/in/josiahhutchinson/',
  '/in/graciela-herrera-9a414b108/',
  '/in/cecetitusinnovations/',
  '/in/andrewpaffrath/',
  '/in/joseph-matysik-ii-4a728092/',
  '/in/stephanie-lopez-966b9390/',
  '/in/brittney-arnold-570369a8/',
  '/in/austinarredondo/',
  '/in/randycourtney/',
  '/in/troygrissom/',
  '/in/alexander-devine-61903540/',
  '/in/tobie-madsen-96887454/',
  '/in/edward-kemper-65929934/',
  '/in/travisamoss/',
  '/in/krystalmires/',
  '/in/everett-davis-835903a9/' ]